<a href="https://colab.research.google.com/github/xloem/techsketball/blob/main/model_import_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

import jax

starting_model_path = 'baffo32/pyc2py_alpha' #'t5-base'#'t5-small'#'bigscience/T0pp'
train_tokenizer = False # baffo32/pyc2py_alpha already has a tokenizer that can output bytes

input_width = 512
# these are not t5 parameters?
train_batch_size = 20 # small for notebook
per_device_batch_size = train_batch_size // jax.device_count()
num_epochs = 1
training_seed = 0
learning_rate = 0.0001#3e-4
logging_steps = 32


#!pip install jax[tpu] -f https://storage.googleapis.com/jax-releases/libtpu_releases.html
!pip3 install deepspeed
!pip3 install transformers
!pip3 install flax
!pip3 install sentencepiece
!git clone https://github.com/xloem/techsketball && ln -s techsketball/* .
!apt-get install git-lfs
!git config --global user.email johndoe@example.com
!git config --global user.name 'John Doe'

     |████████████████████████████████| 515 kB 11.7 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 108 kB 60.4 MB/s 
     |████████████████████████████████| 99 kB 10.1 MB/s 
     |████████████████████████████████| 15.2 MB 49.4 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.5.10-py3-none-any.whl size=528334 sha256=4b2ec24ef925fd2fbe91f72cfe061e2835dd1f63f24a5612c797f69bf684df22
  Stored in directory: /root/.cache/pip/wheels/b9/f1/7e/23e5e8660c7e72626d7dd90f5657d84c7d5c3838c593f82795
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=0d75fddfab38ef3a1663252fa67424509493d2c183eaa9385dd763cb16ffc748
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
Successfully built deepspeed py-cpuinfo
     |████████████████████████████████| 3.4 MB 12.2 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████

In [ ]:
import jax.tools.colab_tpu
import jaxlib
import os
try:
  if 'COLAB_TPU_ADDR' in os.environ:
    jax.tools.colab_tpu.setup_tpu()
  jaxlib.xla_extension.tpu_client()
  backend = 'tpu'
except:
  try:
    jaxlib.xla_extension.gpu_client()
    backend = 'gpu'
  except:
    backend = 'cpu'
import tensorflow as tf
!nvidia-smi
jax.local_devices()

Thu Jan 20 00:18:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    33W / 250W |    257MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[GpuDevice(id=0, process_index=0)]

In [ ]:
from transformers import T5Tokenizer, FlaxT5ForConditionalGeneration 
import huggingface_hub
repo = huggingface_hub.Repository('local_model', clone_from=starting_model_path)
try:
    tokenizer = T5Tokenizer.from_pretrained('local_model')
except:
    tokenizer = T5Tokenizer.from_pretrained(starting_model_path)
try:
    model = FlaxT5ForConditionalGeneration.from_pretrained('local_model')
except:
    model = FlaxT5ForConditionalGeneration.from_pretrained(starting_model_path)

Cloning https://huggingface.co/baffo32/pyc2py_alpha into local empty directory.


Download file flax_model.msgpack:   0%|          | 8.37k/850M [00:00<?, ?B/s]

Download file spiece.model:   2%|1         | 8.37k/515k [00:00<?, ?B/s]

Clean file spiece.model:   0%|          | 1.00k/515k [00:00<?, ?B/s]

Clean file flax_model.msgpack:   0%|          | 1.00k/850M [00:00<?, ?B/s]

In [ ]:
# before data is generated we can import libraries to generate it from
import jax, jax.numpy as jnp
import numpy as np
import optax
import flax
import flax.training.common_utils
import flax.training.train_state
import tqdm
import time
import os
# ...
import transformers
import deepspeed
import scipy
import matplotlib
import sentencepiece
import dis

In [ ]:
import find_pycode
print('getting training data ...')
tokenizerpfx = starting_model_path.replace('/','_') + '.'
find_pycode.write_files('example.', tokenizerpfx, 512, tokenizer, 512, globals(), skip_if_exists = True, train_tokenizer = train_tokenizer, tokenize_binary = True)
tokenizer.save_pretrained('local_model')
# repo.push_to_hub(commit_message=f'commit-message', blocking=False)
train_data = find_pycode.read_files('example.', tokenizerpfx, 512, 512, tokenize_binary = True)

getting training data ...


/usr/local/lib/python3.7/dist-packages/torch/distributed/distributed_c10d.py:171: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  "torch.distributed.reduce_op is deprecated, please use "


In [ ]:
#from tokenizers import ByteLevelBPETokenizer
#tokenizer = ByteLevelBPETokenizer()
#tokenizer.train_from_iterator((str for bytes, str in data_tuples), vocab_size=model.config.vocab_size, min_frequency=2) 

In [ ]:
#cmd_args = None
#model_engine, optimizer, _, _ = deepspeed.initialize(args=cmd_args,
#                                                     model=model,
#                                                     model_parameters=params)

In [ ]:
num_train_steps = len(train_data['input_ids']) // train_batch_size * num_epochs

rng = jax.random.PRNGKey(training_seed)

In [ ]:
def batch_from_indices(dataset : dict, indices):
  #print(dataset['input_ids'].shape, indices.shape)
  result = {
      key : jnp.stack(data[indices,:])
      for key, data in dataset.items()
  }
  # this change could be already put in the dataset passed by the function that produces it
  result['labels'] = result['decoder_input_ids']
  result['decoder_input_ids'] = np.asarray(transformers.models.t5.modeling_flax_t5.shift_tokens_right(result['labels'], tokenizer.pad_token_id, model.config.decoder_start_token_id))
  return result

In [ ]:
# these are not t5 parameters?
linear_decay_lr_schedule_fn = optax.linear_schedule(init_value=learning_rate, end_value=0, transition_steps=num_train_steps)
adamw = optax.adamw(learning_rate=linear_decay_lr_schedule_fn, b1=0.9, b2=0.98, eps=1e-8, weight_decay=0.01)
state = flax.training.train_state.TrainState.create(apply_fn=model.__call__, params=model.params, tx=adamw)

jax.config.update('jax_log_compiles', True)


# from run_t5_mlm_flax.py
dropout_rngs = jax.random.split(rng, jax.local_device_count())

# Define gradient update step fn
def train_step(state, batch, dropout_rng):#input_ids, attention_mask, labels, decoder_input_ids, decoder_attention_mask, dropout_rng):
    dropout_rng, new_dropout_rng = jax.random.split(dropout_rng)
    def loss_fn(params):
        labels = batch.pop('labels')

        #logits = state.apply_fn(
        #    input_ids = input_ids,
        #    attention_mask = attention_mask,
        #    decoder_input_ids = decoder_input_ids,
        #    decoder_attention_mask = decoder_attention_mask,
        #    params = params,
        #    dropout_rng = dropout_rng,
        #    train = True
        #).logits
        logits = state.apply_fn(**batch, params = params, dropout_rng = dropout_rng, train = True).logits
        #print(logits.shape)
        #assert len(logits[-1]) == tokenizer.vocab_size
        #logits = logits[0]

        # logits, labels, padding_mask=batch['decoder_attention_mask', label_smoothing_factor=0]
        # compute loss
        loss = optax.softmax_cross_entropy(logits, flax.training.common_utils.onehot(labels, logits.shape[-1]))
        padding_mask = batch['decoder_attention_mask']
        loss = (loss * padding_mask).sum() / padding_mask.sum()

        #loss = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True).loss

        return loss

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")
    new_state = state.apply_gradients(grads=grad)

    metrics = jax.lax.pmean(
        {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step)}, axis_name="batch"
    )

    return new_state, metrics, new_dropout_rng

# Create parallel version of the train step
p_train_step = jax.pmap(train_step, 'batch', donate_argnums=(0,), backend=backend)

# Replicate the train state on each device
state = flax.jax_utils.replicate(state)

print('Performing initial batch to compile train step ...')
rng, input_rng = jax.random.split(rng)
num_train_samples = len(train_data['input_ids'])
train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
model_inputs = batch_from_indices(train_data, train_samples_idx[:train_batch_size])
model_inputs = flax.training.common_utils.shard(model_inputs)
state, train_metric, dropout_rngs = p_train_step(state, model_inputs, dropout_rng=dropout_rngs)
train_metric = flax.jax_utils.unreplicate(train_metric)
print('Done.  First loss was', train_metric['loss'].mean())

Performing initial batch to compile train step ...


Done.  First loss was 1.6200339


In [ ]:


train_time = 0
epochs = tqdm.tqdm(range(num_epochs), desc="Epoch ... ", position=0)
for epoch in epochs:
    # ======================== Training ================================
    train_start = time.time()
    train_metrics = []

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    num_train_samples = len(train_data['input_ids'])
    train_samples_idx = jax.random.permutation(input_rng, jnp.arange(num_train_samples))
    #train_batch_idx = generate_batch_splits(train_samples_idx, train_batch_size)

    # Gather the indexes for creating the batch and do a training step
    batches_tqdm = tqdm.tqdm(range(num_train_samples // train_batch_size), desc="Training...", position=1)
    for step, batch_idx in enumerate(batches_tqdm):
        #samples = [tokenized_datasets["train"][int(idx)] for idx in batch_idx]
        model_inputs = batch_from_indices(train_data, train_samples_idx[train_batch_size * batch_idx:train_batch_size * batch_idx + train_batch_size])
        #print('model_inputs are', {key:val.shape for key, val in model_inputs.items()})
        #model_inputs = data_collator(samples)

        # Model forward
        model_inputs = flax.training.common_utils.shard(model_inputs)
        state, train_metric, dropout_rngs = p_train_step(state, model_inputs, dropout_rng=dropout_rngs)
        train_metrics.append(train_metric)

        cur_step = epoch * (num_train_samples // train_batch_size) + step

        if cur_step % logging_steps == 0 and cur_step > 0 and jax.process_index() == 0:
            # Save metrics
            train_metric = flax.jax_utils.unreplicate(train_metric)
            train_time += time.time() - train_start
            #if has_tensorboard and jax.process_index() == 0:
            #    write_train_metric(summary_writer, train_metrics, train_time, cur_step)

            epochs.write(
                f"Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()}"
            )

            train_metrics = []
        #if cur_step % (num_train_samples // 8) == 0:
            # save checkpoint
            if jax.process_index() == 0:
                params = jax.device_get(jax.tree_map(lambda x: x[0], state.params))
                model.save_pretrained('local_model', params=params)
                repo.push_to_hub(commit_message=f'step={step} loss={train_metric["loss"].mean()}', blocking=False)

Training...:   2%|▏         | 31/1715 [45:08<35:53:21, 76.72s/it]

Training...:   2%|▏         | 32/1715 [47:42<35:51:01, 76.69s/it]

Loss: 1.8676849603652954, Learning Rate: 0.0002971137291751802



Training...:   4%|▎         | 63/1715 [1:27:19<35:44:09, 77.87s/it]

Training...:   4%|▎         | 64/1715 [1:29:53<35:35:53, 77.62s/it]

Loss: 1.1610958576202393, Learning Rate: 0.00029431487200781703


Several commits (2) will be pushed upstream.

Training...:   6%|▌         | 95/1715 [2:10:37<35:42:20, 79.35s/it]

Training...:   6%|▌         | 96/1715 [2:13:15<35:33:30, 79.07s/it]

...


Training...:  48%|████▊     | 831/1715 [21:20:20<23:28:39, 95.61s/it] 

Training...:  49%|████▊     | 832/1715 [21:24:08<25:21:12, 103.37s/it]

Loss: 1.0832576751708984, Learning Rate: 0.00022714286751579493


Several commits (2) will be pushed upstream.

Training...:  50%|█████     | 863/1715 [22:09:33<20:17:55, 85.77s/it]

Training...:  50%|█████     | 864/1715 [22:12:22<20:15:12, 85.68s/it]

Loss: 0.7977034449577332, Learning Rate: 0.00022434403945226222



Training...:  52%|█████▏    | 889/1715 [22:53:28<22:16:11, 97.06s/it]

In [ ]:
while True:
  print(repr(eval(input('>>> '), globals(), locals())))